In [2]:
import tensorflow as tf

import numpy as np
import os
import time

In [7]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [8]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [9]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [10]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [11]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [12]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [13]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [14]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [15]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [16]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [17]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

## Prediction task

In [18]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [19]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [20]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [21]:
seq_length = 100

In [22]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [23]:
#join keywords back to strings
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [24]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [25]:
# @title
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [26]:
# @title
dataset = sequences.map(split_input_target)

In [27]:
# @title
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


## Create training batches

In [28]:
# @title
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

In [29]:
# @title
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [30]:
# @title
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [5]:
# @title
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

NameError: name 'MyModel' is not defined

Try Model

In [32]:
# @title
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [33]:
# @title
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
# @title
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [35]:
# @title
sampled_indices

array([60,  6, 33, 30, 45, 61, 14, 27, 29, 14, 23, 63, 28, 48, 31, 22, 19,
       14, 18, 27, 14, 64, 35, 44, 64, 50,  1, 17, 32, 47, 49, 34, 19, 54,
       18, 21, 19, 15,  8, 25, 27, 42, 52, 18, 51, 36, 47,  4, 36, 56, 29,
       57, 61, 15, 35, 64, 21, 48, 16, 40, 59,  1, 55, 15, 61, 17, 46, 63,
       57, 30, 62, 15, 15,  7,  2, 11, 42, 14, 26, 41, 54,  9, 28, 52, 40,
       44, 55, 44, 53, 48, 18, 58, 43,  1, 35, 49,  0, 29,  0, 22])

In [36]:
# @title
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' much:\nLord, Lord, she will be a joyful woman.\n\nROMEO:\nWhat wilt thou tell her, nurse? thou dost not'

Next Char Predictions:
 b"u'TQfvANPAJxOiRIFAENAyVeyk\nDShjUFoEHFB-LNcmElWh$WqPrvBVyHiCat\npBvDgxrQwBB, :cAMbo.OmaepeniEsd\nVj[UNK]P[UNK]I"


# Train the model

In [37]:
# @title
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [38]:
# @title
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.190758, shape=(), dtype=float32)


In [39]:
# @title
tf.exp(example_batch_mean_loss).numpy()

66.07287

In [40]:
# @title
model.compile(optimizer='adam', loss=loss)

In [41]:
# @title
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [42]:
# @title
EPOCHS = 30

In [43]:
# @title
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 737s 4s/step - loss: 2.7128
Epoch 2/30
172/172 [==============================] - 729s 4s/step - loss: 1.9798
Epoch 3/30
172/172 [==============================] - 727s 4s/step - loss: 1.6986
Epoch 4/30
172/172 [==============================] - 726s 4s/step - loss: 1.5410
Epoch 5/30
172/172 [==============================] - 727s 4s/step - loss: 1.4452
Epoch 6/30
172/172 [==============================] - 728s 4s/step - loss: 1.3780
Epoch 7/30
172/172 [==============================] - 727s 4s/step - loss: 1.3260
Epoch 8/30
172/172 [==============================] - 728s 4s/step - loss: 1.2815
Epoch 9/30
172/172 [==============================] - 728s 4s/step - loss: 1.2405
Epoch 10/30
172/172 [==============================] - 727s 4s/step - loss: 1.2007
Epoch 11/30
172/172 [==============================] - 727s 4s/step - loss: 1.1611
Epoch 12/30
172/172 [==============================] - 728s 4s/step - loss: 1.1188
Epoch 13/30
1

In [3]:
# @title
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
# @title
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [46]:
# @title
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Then she is in the matter of our hind perforking
Or soldiers shall not might heaven presure to
supper; but the kind senation as I pardon
Forch their way, and me again the second nor clamosome
Where she--mageted to rejoice and stones;
And then, so often hast thou, whose person whate'er had been,
For I love Juliet, a rather both of birch,
The faults of honour, but you wrong'd,
That in my choes world's health to the marnel,
Nor how its country's stranger, being private messenger:
Confined by a credot air, to hear you tell me who
Hoverancies Kate, be bones to the appetite,
To fetch a bad abroad curted,
When I am unking'd brave belona,
A man ado to tear upon our farmic each,
That it great persuage it.

First Watchman:
Under thy mind of what is poisonous
With a bowl that thou hast broke off freesy
Thou lovest as trues,
Woo Against the case! a brother of imprison me?

HASTINGS:
Only, good father, for they are almost friar,
And consequence it out flaits.

GLOUCESTER:
The sun is wounded,

In [47]:
# @title
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Whither?

DORCAS:
I would I keep it: Hearing God warlike prayers!
They are free men, but I cannot get a meeting wit.

BUCKINGHAM:
Marry, my lord, I claim you still. Pence me
She lets that way, and mine art touchest to him:
Yea, since it is some slander of the senate-shines,
And many an honest malice oft;
And that's worthy men! I have trusted him to
the beggar-tate is your followers.

First Lord:
The heavens ghos, the which we have been
Deteit upon him, he is choise unto the crown,
That pain thy gubbers at all presumption.

KING RICHARD III:
Being bo, good sount!

RICHOPFO:
Sir William Lord Hastings and Menner
since: you violaty, the all uncles begun,
And thou a king: plain your queen and Learn'd each one fie!

Provost:
Who calls? keep you himself, for he hath something
deared to hear? Who's he is, he would
son--Gok, my lord, Edward for a king!
Come, thou unpossession that condunt your wave
He title in thy choice: then, farewell
Of hy; thou'rt ill beloved, he
Stands that an augum

In [48]:
# @title
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThe lands red me in the marriage of\nWhich was out to pierce in a foe,\nThough close with height more pitience and deep excales\nTo fortune she is not in England's house;\nHe shall help Clappis Beaugh'd a carencalation and\nBy kingly sweet to be in fashionable as any\ngirth, yet being a king, all eyes.\n\nFirst Citizen:\nThen have you not add more, adieu!\n\nAUTOLYCUS:\nI understand the business! where is he loves,\nTo blot out mine, if the wager thou the beggary's bow:\nSlew whos, say I am to Marcius come all,\nTo give them thine out of change; and let him come hither:\nThe goddess or hold cloud Richmond,\nWhen you have not predect on nature stood,\nAnd speign may deep duly downree again.\n\nROMEO:\nThen, for my sake good speed your head, and be it so;\nInswer'd thus, for my mage in my deeds,\nTo brand my honour'd friendship in them,\nThat do coil him not defend thine own defress,\nA most worthy gentle queen in wrong.\nBut that a hundred men call upon you\nTo par

In [49]:
# @title
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [50]:
# @title
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
You have been in commody that tulls about,
I was another button on:
Let him be false, pounds your h
